## Reducer: (Put your name here)

In [ ]:
from pathlib import Path

from ipyautoui.custom import FileChooser
import ipywidgets as ipw

import reducer.gui
import reducer.astro_gui as astro_gui
from reducer.image_browser import ImageBrowser

from ccdproc import ImageFileCollection

from reducer import __version__
print(__version__)

# Select the directory that contains your data in the cell below

In [ ]:
select_uncalibrated_data = FileChooser(show_only_dirs=True)
select_uncalibrated_data

In [ ]:
data_dir = select_uncalibrated_data.value
print(f'Uncalibrated data is in {data_dir}')

## Your calibrated images will go into a folder called "reduced"

In [ ]:
destination_dir = 'reduced'

path = Path(".") / destination_dir
path.mkdir(exist_ok=True)

# Type any comments about this dataset here

Double-click on the cell to start editing it.

# Load your data set

In [ ]:
images = ImageFileCollection(location=data_dir, keywords='*')

# Image Summary

## Includes browser and image/metadata viewer

This is not, strictly speaking, part of reduction, but is a handy way to take a quick look at your files.

In [ ]:
fits_browser = ImageBrowser(images, keys=['imagetyp', 'exptime'])
fits_browser.display()

# Check names of image types and exposure time keyword

In [ ]:
input_imagetypes = sorted(set(images.summary["imagetyp"]))
types = ["bias", "dark", "flat", "light"]
EXPOSURE_KEYWORDS = ["EXPOSURE", "EXPTIME", "TELAPSE", "ELAPTIME", "ONTIME", "LIVETIME"]

widgets = {}
for ty in types:
    match_types = [in_im for in_im in input_imagetypes if ty in in_im.lower()]
    widgets[ty] = ipw.Dropdown(description=ty.capitalize(), options=input_imagetypes)
    widgets[ty].value = match_types[0] if match_types else input_imagetypes[0]

exp_times_chooser = ipw.Dropdown(description="Exposure time", options=EXPOSURE_KEYWORDS)
maybe_exp_time = [exp for exp in EXPOSURE_KEYWORDS if exp.lower() in images.summary.colnames]
exp_times_chooser.value = maybe_exp_time[0] if maybe_exp_time else EXPOSURE_KEYWORDS[0]

title = ipw.HTML(value="Choose the keyword in your images for each type")
vb = ipw.VBox(children=[title] + list(widgets.values()) + [exp_times_chooser])
vb


In [ ]:
imagetype_map = {k: v.value for k, v in widgets.items()}
exposure_time_keyword = exp_times_chooser.value.lower()

In [ ]:
imagetype_map, exposure_time_keyword

# Make a combined bias image

## Reduce the bias images

In [ ]:
bias_reduction = astro_gui.Reduction(description='Reduce bias frames',
                                     toggle_type='button',
                                     allow_bias=False,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     imagetype_map=imagetype_map,
                                     apply_to={'imagetyp': 'bias'},
                                     destination=destination_dir)
bias_reduction.display()

In [ ]:
print(bias_reduction)

## Combine bias images to make combined bias

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
bias = astro_gui.Combiner(description="Combined Bias Settings",
                          toggle_type='button',
                          file_name_base='combined_bias',
                          image_source=reduced_collection,
                          imagetype_map=imagetype_map,
                          apply_to={'imagetyp': 'bias'},
                          destination=destination_dir)
bias.display()

In [ ]:
print(bias)

# Make a combined dark

## Reduce dark images

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
dark_reduction = astro_gui.Reduction(description='Reduce dark frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=False,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     imagetype_map=imagetype_map,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'dark'})

dark_reduction.display()

In [ ]:
print(dark_reduction)

## Combine reduced darks

Note the ``Group by`` option in the controls that appear after you execute the cell below. `reducer` will make a master for each value of the FITS keyword listed in ``Group by``. By default this keyword is named ``exposure`` for darks, so if you have darks with exposure times of 10 sec, 15 sec and 120 sec you will get three master darks, one for each exposure time.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

dark = astro_gui.Combiner(description="Make Combined Dark(s)",
                          toggle_type='button',
                          file_name_base='combined_dark',
                          group_by=exposure_time_keyword,
                          image_source=reduced_collection,
                          imagetype_map=imagetype_map,
                          apply_to={'imagetyp': 'dark'},
                          destination=destination_dir)
dark.display()

In [ ]:
print(dark)

# Make combined flats

## Reduce flat images

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
flat_reduction = astro_gui.Reduction(description='Reduce flat frames',
                                     toggle_type='button',
                                     allow_bias=True,
                                     master_source=reduced_collection,
                                     allow_dark=True,
                                     allow_flat=False,
                                     input_image_collection=images,
                                     imagetype_map=imagetype_map,
                                     exposure_keyword=exposure_time_keyword,
                                     destination=destination_dir,
                                     apply_to={'imagetyp': 'flat'})

flat_reduction.display()

In [ ]:
print(flat_reduction)

## Build combined flats

Again, note the presence of ``Group by``. If you typically use twilight flats you will almost certainly want to group by filter, not by filter and exposure.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

flat = astro_gui.Combiner(description="Make Combined Flat(s)",
                          toggle_type='button',
                          file_name_base='combined_flat',
                          group_by='filter',
                          image_source=reduced_collection,
                          imagetype_map=imagetype_map,
                          apply_to={'imagetyp': 'flat'},
                          destination=destination_dir)
flat.display()

In [ ]:
print(flat)

# Reduce the science images

There is some autmatic matching going on here:

+ If darks are subtracted a dark of the same edxposure time will be used, if available. If not, and dark scaling is enabled, the dark with the closest exposure time will be scaled to match the science image.
+ If the dark you want to scale appears not to be bias-subtracted an error will be raised.
+ Flats are matched by filter.

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')
light_reduction = astro_gui.Reduction(description='Reduce light frames',
                                      toggle_type='button',
                                      allow_cosmic_ray=True,
                                      master_source=reduced_collection,
                                      input_image_collection=images,
                                      imagetype_map=imagetype_map,
                                      exposure_keyword=exposure_time_keyword,
                                      destination=destination_dir,
                                      apply_to={'imagetyp': 'light'})

light_reduction.display()

## Wonder what the reduced images look like? Make another image browser...

In [ ]:
reduced_collection = ImageFileCollection(location=destination_dir, keywords='*')

In [ ]:
reduced_browser = ImageBrowser(reduced_collection, keys=['imagetyp', 'filter'])
reduced_browser.display()